In [ ]:
def revert_to_the_mean(first data point to trade on, model):
    train model with the last 100 observations
    while next_data_point in the upper 70% quantile:
        train model with the last 100 observation (leaving out the first datapoint and adding the latest)
    buy stock for 5000000
    take new data point and train a model with the latest 100 observations
    while next_data_point in the lower 70% quantile and three days haven't passed or the block has ended:
        hold
    sell sell everything


# In die Strategie das unterteste 10% exkludieren weil das vielleicht sehr starke negative Updates sinds (Trend updates)


def revert_to_the_mean_buy(model, next_data_point):
    if next_data_point in the upper 70% quantile:
        return 1, 5000000*next_data_point
    else:
        return 0, None
                
def revert_to_the_mean_sell(model, investment, next_data_point):
    if next_data_point in the lower 70% quantile:
        return 0, investment*next_data_point
    else:
        return 1, None

def trade(strategy, model):
    for block in blocks:
        mutex = 0
        for i+100 in data_point in block:
            train model with the last 100 observations
            if mutex = 0
                mutex, investment = buy(model, next_data_point)
            if mutex = 1
                mutex, profit = sell(investment, model)
             

        